In [ ]:
from pathlib import Path
import os
import sys
import open3d as o3d
from matplotlib import pyplot as plt
import numpy as np

# Add the src directory to the path so the nerfgym module can be imported
base_dir = Path(os.getcwd()).parent.parent.resolve()
sys.path.append(f"{base_dir / 'src'}")
from nerfgym.NeRFEnv import NeRFEnv

In [ ]:
# Please set session_name to the name of the session you want to use
config_path = base_dir / "outputs/poster/splatfacto/2024-11-15_002955/config.yml"
ply_path = base_dir / "outputs/poster/point_cloud.ply"

In [ ]:
# convert the point cloud to a .pcd file because pcl doesn't support .ply
pcd = o3d.io.read_point_cloud(str(ply_path))
pcd_path = ply_path.with_suffix('.pcd')
o3d.io.write_point_cloud(str(pcd_path), pcd)

In [ ]:
# Load point cloud
pcd = o3d.io.read_point_cloud(str(pcd_path))

# you can now navigate around the point cloud and decide where to crop the spurious points.
o3d.visualization.draw_plotly([pcd], point_sample_factor=0.01)

In [ ]:
# The minimum and maximum bounds of the volume in the x, y, and z directions
min_bound = np.array([-4, -4, -4]) # these values are just example for the poster env
max_bound = np.array([4, 4, 4])

# Crop the point cloud to the volume
vol = o3d.geometry.AxisAlignedBoundingBox(
    min_bound=min_bound, max_bound=max_bound
)
pcd_cropped = pcd.crop(vol)

# Visualize the cropped point cloud
o3d.visualization.draw_plotly([pcd_cropped], point_sample_factor=0.1)

In [ ]:
# Set the base directory to the root of the project due to some assumptions in the code
os.chdir(base_dir)

# Initialise the SplatGym environment
nerf_env = NeRFEnv(
    config_path,
    pcd_path,
    min_bound=min_bound,
    max_bound=max_bound,
    bbox_sides=2, # The size of the playable area
    camera_box_size=[0.01,0.01,0.01]  # The size of the ego camera bounding box
)

In [ ]:
# Let's render the first image
nerf_env.reset()
img = nerf_env.render()
plt.imshow(img)

In [ ]:
# Now you can move around the environment and decide where to set the goal position.
# Actions: 0: forward, 1: backward, 2: left, 3: right
action = 0
img, reward, truncated, done, info = nerf_env.step(0)
print(f"reward {reward}, truncated {truncated}, done {done}, info {info}")
print(f"xyzrpy {nerf_env.vc.xyzrpy()}")
plt.imshow(img)
plt.title("reward: {:.2f}".format(reward))

In [ ]:
nerf_env.goal_xyzrpy = [
    0.3179229490458535,
    0.044831075423480554,
    -0.3,
    90.00000000000004,
    0.0,
    80.0000000000001,
]
# set the step size for going forward and backwards
nerf_env.TRANSLATE_STEP = 0.1
nerf_env.reset()

# transfer all the settings to the training script and you are ready to train